<a href="https://colab.research.google.com/github/SorokinMaksimArtemovich/MTS-ML-CUP/blob/main/data%20preprocessing/0.1_price_and_date_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this nootebook I use price and date for aggregation and making matrix

In [ ]:
import sys
import os
import warnings
os.environ['OPENBLAS_NUM_THREADS'] = '1'
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import gc
import time
import pyarrow as pa
import pyarrow.parquet as pq
import scipy
import implicit
import bisect
import sklearn.metrics as m
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
!pip install feather-format >> none
!pip install faiss-cpu --no-cache
!pip install implicit
import implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 188.0 MB/s eta 0:00:00


# Price aggregation
I aggregate counts and sums of request_cnt by price groups and user and count mean, median, min, max and standard deviation od it by each user

In [ ]:
data = pd.read_feather('/data/dataset_full.feather')
data = data[['user_id', 'price', 'request_cnt']]

In [ ]:
price_median = np.nanmedian(data["price"])
data["price"] = data["price"].fillna(price_median)

In [ ]:
def price_bucket(x):
    return bisect.bisect_left(range(5000, 85001, 5000), x)

In [ ]:
price_df = pd.DataFrame()
price_df['price'] = data['price'].drop_duplicates()
price_df['price_cat'] = price_df['price'].apply(price_bucket)
price_df

,price,price_cat
0,20368.0,4
1550,4990.0,0
2272,74259.0,14
3907,23876.0,4
5477,20465.0,4
...,...,...
322204409,44151.0,8
322498461,87207.0,17
322591672,6090.0,1
322605777,12217.0,2


In [ ]:
data = pa.Table.from_pandas(data)
price_df = pa.Table.from_pandas(price_df)
data = data.join(price_df, 'price', join_type = 'inner')

In [ ]:
del(price_df)
gc.collect()

21

In [ ]:
%%time
price_agg = data.select(['user_id', 'price_cat', 'request_cnt']).\
    group_by(['user_id', 'price_cat']).aggregate([('request_cnt', 'count'), ('request_cnt', 'sum')]).\
    group_by(['user_id']).aggregate([('request_cnt_count', 'mean'), ('request_cnt_count', 'approximate_median'), ('request_cnt_count', 'min'), ('request_cnt_count', 'max'), ('request_cnt_count', 'stddev'), ('request_cnt_sum', 'mean'), ('request_cnt_sum', 'approximate_median'), ('request_cnt_sum', 'min'), ('request_cnt_sum', 'max'), ('request_cnt_sum', 'stddev')])

CPU times: user 13.6 s, sys: 3.48 s, total: 17.1 s
Wall time: 17.1 s


In [ ]:
price_agg.to_pandas().to_csv('data/price_and_date_preprocessing/price_agg.csv', index=False)

In [ ]:
del(price_agg)
gc.collect()

21

In [ ]:
from sklearn.preprocessing import StandardScaler

data = data.to_pandas()
prices = data["price"].values
prices = np.log1p( prices )
scaler = StandardScaler()
scaler.fit( prices.reshape(-1, 1) )
prices = scaler.transform( prices.reshape(-1, 1) )
prices = prices.reshape(-1, )        
data["price"] = prices
data = pa.Table.from_pandas(data)

In [ ]:
del(prices)
del(scaler)
gc.collect()

42

In [ ]:
%%time
price = data.select(['user_id', 'price']).\
    group_by(['user_id']).aggregate([('price', 'mean'), ('price', 'approximate_median'), ('price', 'min'), ('price', 'max'), ('price', 'stddev')])

CPU times: user 27.4 s, sys: 2.57 s, total: 29.9 s
Wall time: 29.8 s


In [ ]:
price.to_pandas().to_csv('data/price_and_date_preprocessing/price.csv', index=False)

In [ ]:
del(price)
del(data)
gc.collect()

63

# Date
I extract months, days_of_month, days_of_week from date and agregate request_cnt by it

In [ ]:
data = pd.read_feather('/data/dataset_full.feather')
data = pa.Table.from_pandas(data)

In [ ]:
data = data.select(['user_id', 'date', 'request_cnt', 'part_of_day']).to_pandas()
data['month'] = data['date'].dt.month
data['dayofmonth'] = data['date'].dt.day
data['dayofweek'] = data['date'].dt.dayofweek
data

,user_id,date,request_cnt,part_of_day,month,dayofmonth,dayofweek
0,45098,2022-06-15,1,morning,6,15,2
1,45098,2022-06-19,1,morning,6,19,6
2,45098,2022-06-12,1,day,6,12,6
3,45098,2022-05-16,1,day,5,16,0
4,45098,2022-05-30,1,day,5,30,0
...,...,...,...,...,...,...,...
322899430,300964,2021-07-12,1,morning,7,12,0
322899431,300964,2021-06-20,1,evening,6,20,6
322899432,300964,2021-08-05,1,day,8,5,3
322899433,300964,2021-07-19,1,evening,7,19,0


In [ ]:
data = pa.Table.from_pandas(data)

# Matrix
I encode this features with count or sum of request_cnt

### month

In [ ]:
%%time
month_agg = data.select(['user_id', 'month', 'request_cnt']).\
    group_by(['user_id', 'month']).aggregate([('request_cnt', 'count')])

CPU times: user 12.2 s, sys: 816 ms, total: 13 s
Wall time: 13 s


In [ ]:
month_set = set(month_agg.select(['month']).to_pandas()['month'])
print(f'{len(month_set)} months')
month_dict = {url: idurl for url, idurl in zip(month_set, range(len(month_set)))}
usr_set = set(month_agg.select(['user_id']).to_pandas()['user_id'])
print(f'{len(usr_set)} users')
usr_dict = {usr: user_id for usr, user_id in zip(usr_set, range(len(usr_set)))}

12 months
415317 users


In [ ]:
values = np.array(month_agg.select(['request_cnt_count']).to_pandas()['request_cnt_count'])
rows = np.array(month_agg.select(['user_id']).to_pandas()['user_id'].map(usr_dict))
cols = np.array(month_agg.select(['month']).to_pandas()['month'].map(month_dict))
mat = scipy.sparse.coo_matrix((values, (rows, cols)), shape=(rows.max() + 1, cols.max() + 1)).toarray()

In [ ]:
%%time
inv_usr_map = {v: k for k, v in usr_dict.items()}
month_mat = pd.DataFrame(mat)
month_mat['user_id'] = month_mat.index.map(inv_usr_map)
month_mat.to_csv('data/price_and_date_preprocessing/month_count.csv', index=False)

CPU times: user 1.57 s, sys: 83.7 ms, total: 1.65 s
Wall time: 1.6 s


In [ ]:
del(month_agg)
del(month_set)
del(month_dict)
del(usr_set)
del(usr_dict)
del(values)
del(rows)
del(cols)
del(mat)
del(inv_usr_map)
del(month_mat)
gc.collect()

42

### dayofmonth

In [ ]:
%%time
dayofmonth_agg = data.select(['user_id', 'dayofmonth', 'request_cnt']).\
    group_by(['user_id', 'dayofmonth']).aggregate([('request_cnt', 'count')])

CPU times: user 14.3 s, sys: 1.17 s, total: 15.4 s
Wall time: 15.4 s


In [ ]:
dayofmonth_set = set(dayofmonth_agg.select(['dayofmonth']).to_pandas()['dayofmonth'])
print(f'{len(dayofmonth_set)} daysofmonth')
dayofmonth_dict = {url: idurl for url, idurl in zip(dayofmonth_set, range(len(dayofmonth_set)))}
usr_set = set(dayofmonth_agg.select(['user_id']).to_pandas()['user_id'])
print(f'{len(usr_set)} users')
usr_dict = {usr: user_id for usr, user_id in zip(usr_set, range(len(usr_set)))}

31 daysofmonth
415317 users


In [ ]:
%%time
values = np.array(dayofmonth_agg.select(['request_cnt_count']).to_pandas()['request_cnt_count'])
rows = np.array(dayofmonth_agg.select(['user_id']).to_pandas()['user_id'].map(usr_dict))
cols = np.array(dayofmonth_agg.select(['dayofmonth']).to_pandas()['dayofmonth'].map(dayofmonth_dict))
mat = scipy.sparse.coo_matrix((values, (rows, cols)), shape=(rows.max() + 1, cols.max() + 1)).toarray()

CPU times: user 851 ms, sys: 259 ms, total: 1.11 s
Wall time: 1.09 s


In [ ]:
%%time
inv_usr_map = {v: k for k, v in usr_dict.items()}
dayofmonth_mat = pd.DataFrame(mat)
dayofmonth_mat['user_id'] = dayofmonth_mat.index.map(inv_usr_map)
dayofmonth_mat.to_csv('data/price_and_date_preprocessing/dayofmonth_count.csv', index=False)

CPU times: user 2.78 s, sys: 81.2 ms, total: 2.86 s
Wall time: 2.85 s


In [ ]:
del(dayofmonth_agg)
del(dayofmonth_set)
del(dayofmonth_dict)
del(usr_set)
del(usr_dict)
del(values)
del(rows)
del(cols)
del(mat)
del(inv_usr_map)
del(dayofmonth_mat)
gc.collect()

21

### dayofweek

In [ ]:
%%time
dayofweek_agg = data.select(['user_id', 'dayofweek', 'request_cnt']).\
    group_by(['user_id', 'dayofweek']).aggregate([('request_cnt', 'sum')])

CPU times: user 12.8 s, sys: 877 ms, total: 13.6 s
Wall time: 13.6 s


In [ ]:
dayofweek_set = set(dayofweek_agg.select(['dayofweek']).to_pandas()['dayofweek'])
print(f'{len(dayofweek_set)} dayofweeks')
dayofweek_dict = {url: idurl for url, idurl in zip(dayofweek_set, range(len(dayofweek_set)))}
usr_set = set(dayofweek_agg.select(['user_id']).to_pandas()['user_id'])
print(f'{len(usr_set)} users')
usr_dict = {usr: user_id for usr, user_id in zip(usr_set, range(len(usr_set)))}

7 dayofweeks
415317 users


In [ ]:
%%time
values = np.array(dayofweek_agg.select(['request_cnt_sum']).to_pandas()['request_cnt_sum'])
rows = np.array(dayofweek_agg.select(['user_id']).to_pandas()['user_id'].map(usr_dict))
cols = np.array(dayofweek_agg.select(['dayofweek']).to_pandas()['dayofweek'].map(dayofweek_dict))
mat = scipy.sparse.coo_matrix((values, (rows, cols)), shape=(rows.max() + 1, cols.max() + 1)).toarray()

CPU times: user 538 ms, sys: 77.4 ms, total: 616 ms
Wall time: 599 ms


In [ ]:
%%time
inv_usr_map = {v: k for k, v in usr_dict.items()}
dayofweek_mat = pd.DataFrame(mat)
dayofweek_mat['user_id'] = dayofweek_mat.index.map(inv_usr_map)
dayofweek_mat.to_csv('data/price_and_date_preprocessing/dayofweek_sum.csv', index=False)

CPU times: user 1.34 s, sys: 68.8 ms, total: 1.41 s
Wall time: 1.41 s


In [ ]:
del(dayofweek_agg)
del(dayofweek_set)
del(dayofweek_dict)
del(usr_set)
del(usr_dict)
del(values)
del(rows)
del(cols)
del(mat)
del(inv_usr_map)
del(dayofweek_mat)
gc.collect()

21

## AGG
I aggregate counts and sums of request_cnt by features and user and count mean, median, min, max and standard deviation od it by each user

### month

In [ ]:
%%time
month_agg = data.select(['user_id', 'month', 'request_cnt']).\
    group_by(['user_id', 'month']).aggregate([('request_cnt', 'count'), ('request_cnt', 'sum')]).\
    group_by(['user_id']).aggregate([('request_cnt_count', 'mean'), ('request_cnt_count', 'approximate_median'), ('request_cnt_count', 'min'), ('request_cnt_count', 'max'), ('request_cnt_count', 'stddev'), ('request_cnt_sum', 'mean'), ('request_cnt_sum', 'approximate_median'), ('request_cnt_sum', 'min'), ('request_cnt_sum', 'max'), ('request_cnt_sum', 'stddev')])

CPU times: user 17.3 s, sys: 12.2 s, total: 29.6 s
Wall time: 29.6 s


In [ ]:
month_agg.to_pandas().to_csv('data/price_and_date_preprocessing/month_agg.csv', index=False)

In [ ]:
del(month_agg)
gc.collect()

63

### day_of_month

In [ ]:
%%time
dayofmonth_agg = data.select(['user_id', 'dayofmonth', 'request_cnt']).\
    group_by(['user_id', 'dayofmonth']).aggregate([('request_cnt', 'count'), ('request_cnt', 'sum')]).\
    group_by(['user_id']).aggregate([('request_cnt_count', 'mean'), ('request_cnt_count', 'approximate_median'), ('request_cnt_count', 'min'), ('request_cnt_count', 'max'), ('request_cnt_count', 'stddev'), ('request_cnt_sum', 'mean'), ('request_cnt_sum', 'approximate_median'), ('request_cnt_sum', 'min'), ('request_cnt_sum', 'max'), ('request_cnt_sum', 'stddev')])

CPU times: user 21.5 s, sys: 3.42 s, total: 24.9 s
Wall time: 24.7 s


In [ ]:
dayofmonth_agg.to_pandas().to_csv('data/price_and_date_preprocessing/dayofmonth_agg.csv', index=False)

In [ ]:
del(dayofmonth_agg)
gc.collect()

63

### day_of_week

In [ ]:
%%time
dayofweek_agg = data.select(['user_id', 'dayofweek', 'request_cnt']).\
    group_by(['user_id', 'dayofweek']).aggregate([('request_cnt', 'count'), ('request_cnt', 'sum')]).\
    group_by(['user_id']).aggregate([('request_cnt_count', 'mean'), ('request_cnt_count', 'approximate_median'), ('request_cnt_count', 'min'), ('request_cnt_count', 'max'), ('request_cnt_count', 'stddev'), ('request_cnt_sum', 'mean'), ('request_cnt_sum', 'approximate_median'), ('request_cnt_sum', 'min'), ('request_cnt_sum', 'max'), ('request_cnt_sum', 'stddev')])

CPU times: user 15.9 s, sys: 1.01 s, total: 16.9 s
Wall time: 16.7 s


In [ ]:
dayofweek_agg.to_pandas().to_csv('data/price_and_date_preprocessing/dayofweek_agg.csv', index=False)

In [ ]:
del(dayofweek_agg)
gc.collect()

63

### day_of_month and part_of_day

In [ ]:
%%time
dayofweek_agg = data.select(['user_id', 'dayofweek', 'request_cnt', 'part_of_day']).\
    group_by(['user_id', 'dayofweek', 'part_of_day']).aggregate([('request_cnt', 'count'), ('request_cnt', 'sum')]).\
    group_by(['user_id']).aggregate([('request_cnt_count', 'mean'), ('request_cnt_count', 'approximate_median'), ('request_cnt_count', 'min'), ('request_cnt_count', 'max'), ('request_cnt_count', 'stddev'), ('request_cnt_sum', 'mean'), ('request_cnt_sum', 'approximate_median'), ('request_cnt_sum', 'min'), ('request_cnt_sum', 'max'), ('request_cnt_sum', 'stddev')])

CPU times: user 21.8 s, sys: 1.91 s, total: 23.7 s
Wall time: 23.4 s


In [ ]:
dayofweek_agg.to_pandas().to_csv('data/price_and_date_preprocessing/dayofweek_with_part_of_day_agg.csv', index=False)

In [ ]:
del(data)
del(dayofweek_agg)
gc.collect()

63